In [1]:
!pip install transformers datasets -q

* Hugging Face Datasets 라이브러리에서 한국어 벤치마크인 KLUE(Korean Language Understanding Evaluation)의 YNAT(뉴스 토픽 분류) 서브셋을 불러옴
    * 라벨(주제) 체계 — 7개 클래스(IT과학(0), 경제(1), 사회(2), 생활문화(3), 세계(4), 스포츠(5), 정치(6))
    * 각 샘플은 title(뉴스 제목) 과 label(주제 ID) 로 이뤄져 있고, 그외 date, guid, url 같은 메타데이터 포함
    * https://arxiv.org/pdf/2105.09680
    * https://github.com/KLUE-benchmark/KLUE

In [2]:
from datasets import load_dataset
dataset = load_dataset('klue', 'ynat')
dataset

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

ynat/train-00000-of-00001.parquet:   0%|          | 0.00/4.17M [00:00<?, ?B/s]

ynat/validation-00000-of-00001.parquet:   0%|          | 0.00/847k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45678 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9107 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date'],
        num_rows: 45678
    })
    validation: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date'],
        num_rows: 9107
    })
})

In [ ]:
# train(45,678행): 모델이 학습(파라미터 업데이트)을 위해 보는 데이터.
# validation (9,107행): 학습 중간중간 검증(하이퍼파라미터 튜닝, 에폭/임계값 결정, 모델 선택)

# 데이터 구조 확인하기

dataset['train'][0]

{'guid': 'ynat-v1_train_00000',
 'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영',
 'label': 3,
 'url': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=227&oid=001&aid=0008508947',
 'date': '2016.06.30. 오전 10:36'}

In [ ]:
dataset["train"].features["label"].names

['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import save_npz
import numpy as np

# 학습/검증 텍스트·라벨 추출
X_tr = [t.strip() for t in dataset["train"]["title"]]
y_tr = dataset["train"]["label"]
X_va = [t.strip() for t in dataset["validation"]["title"]]
y_va = dataset["validation"]["label"]

# 라벨 이름(한글 카테고리)
label_names = dataset["train"].features["label"].names

pd.DataFrame({"text": X_tr, "label": y_tr}).to_csv("ynat_train.csv", index=False)
pd.DataFrame({"text": X_va, "label": y_va}).to_csv("ynat_valid.csv", index=False)
pd.Series(label_names).to_csv("ynat_label_names.txt", index=False, header=False)

In [4]:
vec = TfidfVectorizer(ngram_range=(1,2), min_df=2, max_df=0.95, sublinear_tf=True)

Xtr_tfidf = vec.fit_transform(X_tr).astype(np.float32)
Xva_tfidf = vec.transform(X_va).astype(np.float32)

save_npz("X_train_tfidf.npz", Xtr_tfidf)
save_npz("X_valid_tfidf.npz", Xva_tfidf)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import joblib

from google.colab import drive
drive.mount('/content/drive')

np.random.seed(7777)

Mounted at /content/drive


In [6]:
RESULTS = []          # 각 모델 실행 후 성능 누적
PREDS   = {}          # {name: (estimator, y_pred)}

# 평가 함수 생성

def train_eval_one(estimator, name, Xtr, ytr, Xva, yva, label_names=None, save_path=None, report=False):
    est = estimator.fit(Xtr, ytr)
    pred = est.predict(Xva)
    acc  = accuracy_score(yva, pred)
    f1m  = f1_score(yva, pred, average="macro")
    print(f"[{name.upper()}] Accuracy={acc:.4f}  Macro-F1={f1m:.4f}")
    if report:
        print("\n[Classification Report]")
        print(classification_report(yva, pred, target_names=label_names, digits=4))
        print("Confusion Matrix:\n", confusion_matrix(yva, pred))
    RESULTS.append({"model": name, "accuracy": acc, "macro_f1": f1m})
    PREDS[name] = (est, pred)
    if save_path:
        joblib.dump(est, save_path)
        print("Saved:", save_path)
    return est, pred

* 로지스틱 회귀 (Logistic Regression)

    * 개념: 선형 결합 w·x+b를 시그모이드/소프트맥스에 넣어 확률을 출력하는 선형 분류기.

    * 특징: 해석 용이(가중치 방향/세기), TF-IDF와 궁합 좋음.

    * 장점: 강력한 베이스라인, 확률적 출력으로 임계값 조정/불확실성 해석 가능.

    * 단점: 비선형 경계엔 약함(단, n-gram으로 부분 보완).

    * 핵심 하이퍼파라미터: C(규제 세기; 높을수록 규제 약함), class_weight(불균형 시 "balanced").

In [14]:
# 로지스틱 회귀

_ = train_eval_one(
    LogisticRegression(max_iter=1000), "lr",
    Xtr_tfidf, y_tr, Xva_tfidf, y_va,
    label_names=label_names,
    save_path=None, # 예: "/content/drive/MyDrive/ynat_nb.joblib"
    report=False # True로 바꾸면 리포트/혼동행렬 출력
)

[LR] Accuracy=0.6890  Macro-F1=0.6936


* 나이브 베이즈 (Naive Bayes)

    * 개념: 단어(특징) 간 조건부 독립 가정(naive) + 베이즈 정리로 클래스 확률을 계산하는 확률 분류기.

    * 특징: 매우 빠름, 적은 데이터에서도 안정적으로 작동. 희소한 BoW/TF-IDF와 궁합이 좋음.

    * 장점: 학습/예측 속도 빠름, 구현 간단, 과적합에 비교적 강함.

    * 단점: “단어 독립” 가정이 현실과 달라 복잡한 문맥 반영이 약함.

    * 핵심 하이퍼파라미터: alpha(라플라스 스무딩; 0.1~1.0 사이 탐색 추천).

In [15]:
# 나이브 베이즈

_ = train_eval_one(
    MultinomialNB(), "nb",
    Xtr_tfidf, y_tr, Xva_tfidf, y_va,
    label_names=label_names,
    save_path=None,    # 예: "/content/drive/MyDrive/ynat_nb.joblib"
    report=False       # True로 바꾸면 리포트/혼동행렬 출력
)

[NB] Accuracy=0.6141  Macro-F1=0.6362


* 선형 SVM (Linear SVC)

    * 개념: 고차원 공간에서 클래스 간 마진(거리)을 최대화하는 분리 초평면을 학습.

    * 특징: 희소 고차원(BoW/TF-IDF)에서 매우 강함, 실무 기준 탑티어 베이스라인.

    * 장점: 적은 데이터에서도 성능 좋음, 잡음에 비교적 강함.

    * 단점: 기본 LinearSVC는 확률 출력 없음(필요 시 CalibratedClassifierCV로 보정).

In [9]:
# 선형 SVM

_ = train_eval_one(
    LinearSVC(), "svm",
    Xtr_tfidf, y_tr, Xva_tfidf, y_va,
    label_names=label_names,
    save_path=None,
    report=False
)


[SVM] Accuracy=0.6901  Macro-F1=0.6943


* 결정나무 (Decision Tree)

    * 개념: 특징값 기준 Yes/No **분기(split)**를 반복해 규칙 트리를 학습.

    * 특징: 트리 경로가 해석 직관적, 규칙 예시를 설명하기 좋음.

    * 장점: 모델 구조가 눈으로 보임(시각화), 전처리 요구 낮음.

    * 단점: 텍스트처럼 고차원 희소일 때 과적합/성능 저하가 잦음.

In [10]:
# 의사결정나무

_ = train_eval_one(
    DecisionTreeClassifier(random_state=7777), "dt",
    Xtr_tfidf, y_tr, Xva_tfidf, y_va,
    label_names=label_names,
    save_path=None,
    report=False
)


[DT] Accuracy=0.5439  Macro-F1=0.5634


* 랜덤포레스트 (Random Forest)

    * 개념: 부트스트랩 샘플 + 무작위 특징 하위집합으로 여러 결정나무를 학습해 앙상블.

    * 특징: 단일 트리보다 안정적, 과적합 완화.

    * 장점: 적당한 성능 + 특징 중요도로 해석(어떤 단어가 기여했는지) 가능.

    * 단점: SVM/LR 대비 텍스트 고차원에서 성능이 덜 나올 수 있음, 속도/메모리 부담.

    * 핵심 하이퍼파라미터: n_estimators(트리 수), max_depth, max_features.

In [16]:
# 랜덤포레스트

_ = train_eval_one(
    RandomForestClassifier(n_estimators=300, random_state=7777, n_jobs=-1), "rf",
    Xtr_tfidf, y_tr, Xva_tfidf, y_va,
    label_names=label_names,
    save_path=None,
    report=False
)


[RF] Accuracy=0.6193  Macro-F1=0.6366


* Accuracy (정확도)
    * 정의: 전체 표본 중 맞춘 비율(정답/전체)
    * 장점: 직관적이고 한눈에 이해됨
    * 주의: 클래스 불균형이 크면 착시가 생김

* Precision(정밀도): 내가 그 클래스라고 예측한 것 중 실제로 그 클래스인 비율(TP/(TP+FP))
* Recall(재현율): 실제로 그 클래스인 것 중 잘 맞춘 비율(TP/(TP+FN))
* F1-score: Precision과 Recall의 조화평균(둘 다 높아야 커짐)(2xPxR/(P+R))
* support: 해당 클래스의 실제 표본 수(리포트 마지막 열)
-----------
* Macro-F1 / Weighted-F1 / Micro-F1 (다중분류 평균 방식)
    * Macro-F1: 각 클래스의 F1을 그냥 평균
        * 클래스 불균형에 강함(소수 클래스 영향 반영)
        * 여기서 summary는 Macro-F1 기준 내림차순

* Weighted-F1: 각 클래스 F1을 support로 가중 평균
    * 표본이 많은 클래스의 영향이 큼 → 전체 경향을 반영

* Micro-F1: 다중분류에서 전체 TP/FP/FN을 합산해 F1 계산 (= Micro-Precision = Micro-Recall)
    * 표본 수가 큰 클래스의 영향이 매우 큼
---------------
* Confusion Matrix (혼동행렬)
    * 형태: 행 = 정답, 열 = 예측
    * 해석: 대각선이 클수록 좋음
--------------
* 데이터가 균형: Accuracy, (참고로 Macro-F1)
* 불균형/소수 클래스 중요: Macro-F1 필수, 클래스별 Precision/Recall 체크
* 전체 한 수치로 대략 보고 싶다: Weighted-F1, Accuracy
* 에러 원인 파악: Confusion Matrix로 어느 클래스끼리 섞이는지 확인

In [ ]:
# 요약
summary = pd.DataFrame(RESULTS).sort_values("macro_f1", ascending=False).reset_index(drop=True)
print("\n=== Summary (sorted by Macro-F1) ===")
print(summary)

# 최고 모델 상세 리포트
if not summary.empty:
    best_name = summary.loc[0, "model"]
    est, pred = PREDS[best_name]
    print("\nBest model:", best_name)
    print("\n[Classification Report — best]")
    print(classification_report(y_va, pred, target_names=label_names, digits=4))
    print("Confusion Matrix:\n", confusion_matrix(y_va, pred))


=== Summary (sorted by Macro-F1) ===
  model  accuracy  macro_f1
0   svm  0.690128  0.694276
1    lr  0.689030  0.693581
2    rf  0.619304  0.636572
3    nb  0.614143  0.636182
4    dt  0.543867  0.563449

Best model: svm

[Classification Report — best]
              precision    recall  f1-score   support

        IT과학     0.5706    0.7365    0.6430       554
          경제     0.6970    0.7441    0.7198      1348
          사회     0.8190    0.5796    0.6788      3701
        생활문화     0.6614    0.7589    0.7068      1369
          세계     0.5653    0.7365    0.6396       835
         스포츠     0.7603    0.8945    0.8219       578
          정치     0.5608    0.7729    0.6500       722

    accuracy                         0.6901      9107
   macro avg     0.6621    0.7461    0.6943      9107
weighted avg     0.7147    0.6901    0.6901      9107

Confusion Matrix:
 [[ 408   32   57   28   12    8    9]
 [  67 1003  138   37   48   20   35]
 [ 156  321 2145  396  299   71  313]
 [  51   36  11

In [ ]:
### 위 결과 해석하는 법

### 전체 성능(요약)
# 최고 모델: SVM
# Accuracy 0.6901, Macro-F1 0.6943
# 다중분류에서 클래스 간 균형을 보는 Macro-F1 기준 1위
# LR와 근소한 차이(Macro-F1 0.6936): 선형 계열 두 모델이 텍스트에서 강력한 베이스라인
# RF/NB/DT는 이번 세팅에선 뒤처짐(고차원 희소 특징에서 선형 모델이 유리

### 클래스별 지표(Precision/Recall/F1)
# 괄호는 (정밀도/재현율/F1, support).
# 스포츠(5): 0.760 / 0.895 / 0.822, n=578 → 가장 안정적. 놓치는 비율이 매우 낮음(재현율↑).
# 생활문화(3): 0.661 / 0.759 / 0.707, n=1369 → 전체적으로 균형 양호.
# 경제(1): 0.697 / 0.744 / 0.720, n=1348 → 무난한 클래스.
# 사회(2): 0.819 / 0.580 / 0.679, n=3701 → 정밀도↑, 재현율↓. “사회라고 예측하면 대부분 맞지만, 정답 ‘사회’를 많이 놓침.”
# 정치(6): 0.561 / 0.773 / 0.650, n=722 → 재현율↑, 정밀도↓. 많이 잡아오되 오탐이 섞임.
# IT과학(0): 0.571 / 0.737 / 0.643, n=554 → 정밀도 낮고 재현율 높음(오탐 존재).
# 세계(4): 0.565 / 0.736 / 0.640, n=835 → IT과학과 비슷한 패턴.
# 요약: 사회(2)는 재현율이 낮아(0.58) 많이 놓치고, 정치(6)/IT과학(0)/세계(4)는 정밀도가 낮음

### 3) Confusion Matrix로 본 주요 오류 흐름(행=정답, 열=예측)
# 사회(2) → 생활문화(3): 396건
# 사회(2) → 경제(1): 321건
# 사회(2) → 정치(6): 313건
# 사회(2) → 세계(4): 299건
# 사회가 생활문화/경제/정치/세계와 광범위하게 경계가 겹침.
# 경제(1) → 사회(2): 138건
# 정치(6) → 사회(2): 83건
# IT과학(0) → 사회(2): 57건
# 사회(2)가 다른 주제를 많이 흡수하거나 내뱉는 패턴 다수 포착됨
# 반대로 스포츠(5)는 정답 578 중 517건 정확(재현율 0.895)으로 분류 성능을 높이는 키워드가 있는 것으로 보임

* 아래는 위 단계를 한꺼번에 돌리는 과정 / 결과는 같음

In [28]:
models = {
    "nb": MultinomialNB(),                                 # 가볍고 빠른 베이스라인
    "lr": LogisticRegression(max_iter=1000),               # 선형 모델(확률 미지원 X)
    "svm": LinearSVC(),                                    # 선형 SVM(텍스트에 강함)
    "dt": DecisionTreeClassifier(random_state=7777),       # 단일 트리
    "rf": RandomForestClassifier(n_estimators=300, random_state=7777, n_jobs=-1),  # 앙상블
}

results = []
preds_dict = {}

for name, clf in models.items():
    clf.fit(Xtr_tfidf, y_tr)
    pred = clf.predict(Xva_tfidf)
    preds_dict[name] = (clf, pred)
    acc = accuracy_score(y_va, pred)
    f1m = f1_score(y_va, pred, average="macro")
    results.append({"model": name, "accuracy": acc, "macro_f1": f1m})
    print(f"[{name.upper()}] Accuracy={acc:.4f} Macro-F1={f1m:.4f}")

[NB] Accuracy=0.6141 Macro-F1=0.6362
[LR] Accuracy=0.6890 Macro-F1=0.6936
[SVM] Accuracy=0.6901 Macro-F1=0.6943
[DT] Accuracy=0.5439 Macro-F1=0.5634
[RF] Accuracy=0.6193 Macro-F1=0.6366


In [ ]:
# 요약 테이블 (Macro-F1 내림차순)
summary = pd.DataFrame(results).sort_values("macro_f1", ascending=False).reset_index(drop=True)
print("\n=== Summary (sorted by Macro-F1) ===")
print(summary)

# 베스트 모델 상세 리포트 & 혼동행렬
best_name = summary.loc[0, "model"]
best_clf, best_pred = preds_dict[best_name]
print("\nBest model:", best_name)

print("\n[Classification Report — best]")
print(classification_report(y_va, best_pred, target_names=label_names, digits=4))

cm = confusion_matrix(y_va, best_pred)
print("Confusion Matrix:\n", cm)

# 가장 많이 헷갈린 페어 상위 5개 출력
err = []
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        if i != j and cm[i, j] > 0:
            err.append((cm[i, j], i, j))
if err:
    print("\nTop-5 confusions (True → Pred: count):")
    for c, i, j in sorted(err, reverse=True)[:5]:
        print(f"{label_names[i]} → {label_names[j]}: {c}")

# 베스트 모델 저장
best_path = f"/content/drive/MyDrive/ynat_best_{best_name}.joblib"
joblib.dump(best_clf, best_path)
print("\nSaved best model to:", best_path)


=== Summary (sorted by Macro-F1) ===
  model  accuracy  macro_f1
0   svm  0.690128  0.694276
1    lr  0.689030  0.693581
2    rf  0.619304  0.636572
3    nb  0.614143  0.636182
4    dt  0.543867  0.563449

Best model: svm

[Classification Report — best]
              precision    recall  f1-score   support

        IT과학     0.5706    0.7365    0.6430       554
          경제     0.6970    0.7441    0.7198      1348
          사회     0.8190    0.5796    0.6788      3701
        생활문화     0.6614    0.7589    0.7068      1369
          세계     0.5653    0.7365    0.6396       835
         스포츠     0.7603    0.8945    0.8219       578
          정치     0.5608    0.7729    0.6500       722

    accuracy                         0.6901      9107
   macro avg     0.6621    0.7461    0.6943      9107
weighted avg     0.7147    0.6901    0.6901      9107

Confusion Matrix:
 [[ 408   32   57   28   12    8    9]
 [  67 1003  138   37   48   20   35]
 [ 156  321 2145  396  299   71  313]
 [  51   36  11

In [36]:
from pathlib import Path
from joblib import dump

save_dir = Path("/content/")

dump(vec, save_dir / "tfidf.joblib")

# 모델들 저장(여기서는 3개만 저장)
svm_clf = LinearSVC().fit(Xtr_tfidf, y_tr)
lr_clf  = LogisticRegression(max_iter=1000).fit(Xtr_tfidf, y_tr)
nb_clf  = MultinomialNB().fit(Xtr_tfidf, y_tr)

dump(vec, save_dir / "tfidf.joblib")
dump(svm_clf, save_dir / "svm.joblib")
dump(lr_clf,  save_dir / "lr.joblib")
dump(nb_clf,  save_dir / "nb.joblib")

['/content/nb.joblib']

In [11]:
df = pd.read_csv('/content/drive/MyDrive/Colab/공정성/df100_final1524 v2.csv', encoding='utf-8', lineterminator='\n')
df1 = df.sample(2000)
df1 = df1.reset_index()

df1

,index,Date,Url,Title,Content,News,Style,Year,Char,Nouns,Nouns_cleaned,Admin,Period\r
0,2050,2019.11.28,https://www.chosun.com/site/data/html_dir/2019...,"[단독] '겨울왕국2' 스크린 2648개↑점유율 79%→디즈니 ""공식입장無""..무법...",[OSEN=김보라 기자] 영화 ‘겨울왕국2’가 1편 ‘겨울왕국’(2014)에 이어 ...,조선,Con,2019,160,김보라 기자 영화 겨울왕국 겨울왕국 관객 돌파 모양새 관객 돌파 평일 일일 관객 극...,김보라 영화 겨울왕국 겨울왕국 관객 돌파 모양새 관객 돌파 평일 일일 관객 극장 겨...,Pro,5
1,9626,2018-09-21,https://www.donga.com/news/article/all/2018092...,서울대 총장 재선거에 9명 출사표…오세정 의원 등 등록,바른미래당 오세정 의원을 포함한 9명이 서울대학교 총장 재선거에 출사표를 던졌다. ...,동아,Con,2018,190,미래 오세정 의원 포함 서울대학교 총장 선거 출사표 서울대 총장 최종 후보 성희롱 ...,미래 오세정 의원 포함 서울대학교 총장 선거 출사표 서울대 총장 최종 후보 성희롱 ...,Pro,4
2,16339,2024-02-28,https://www.hani.co.kr/arti/society/media/1130...,법원도 ‘청부 민원’ 부당성 인정…방심위 안팎 “류희림 수사해야”,법원이 야권 추천 심의위원에 대한 ‘류희림 방송통신심의위원회(방심위)’와 윤석열 대...,한겨레,Pro,2024,370,법원 야권 추천 심의 위원 류희 방송 통신 심의 위원회 방심위 윤석열 대통령 해촉 ...,법원 야권 추천 심의 위원 류희 방송 통신 심의 위원회 방심위 윤석열 대통령 해촉 ...,Con,10
3,6923,2021-07-01,https://www.donga.com/news/article/all/2021070...,공식 등판한 이재명 “억강부약…모두 잘 사는 대동세상 향해가겠다”,더불어민주당 대선 주자인 이재명 경기도지사가 1일 온라인을 통해 제20대 대통령 선...,동아,Con,2021,201,더불어민주당 대선 주자 이재명 경기 도지사 온라인 대통령 선거 출마 선언 이재명 캠...,더불어민주당 대선 주자 이재명 경기 도지사 온라인 대통령 선거 출마 선언 이재명 캠...,Pro,7
4,11654,2023-12-06,https://www.donga.com/news/Society/article/all...,채용 과정 개입 후 응시해 팀장 임용…‘채용비리’ 867건 적발,정승윤 국민권익위원회 부위원장 겸 사무처장. 2023.11.22 뉴스1국민권익위원회...,동아,Con,2023,247,정승 국민 권익 위원회 부위원장 사무처장 뉴스1 국민 권익 위원회 공직 유관 단체 ...,정승 국민 권익 위원회 부위원장 사무처장 국민 권익 위원회 공직 유관 단체 공정 채...,Con,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,9235,2019-03-04,https://www.donga.com/news/article/all/2019030...,"성인 된 2000년생, ‘공정세대’가 온다",자신과 연관된 불공정에 분노… “큰 성공 어려워” 소박한 삶 지향\r사람 사귈 때도...,동아,Con,2019,315,자신 연관 공정 분노 성공 지향 사람 가성비 동아일보 기성세대 사이 원활 소통 웹뉴...,연관 공정 분노 성공 지향 가성비 동아일보 기성세대 사이 원활 소통 웹뉴 웹툰 뉴스...,Pro,5
1996,9358,2019-01-08,https://www.donga.com/news/article/all/2019010...,‘공익위원 추천 정부 독점’ 없앴지만 노사 갈등 더 커질 우려,[최저임금 결정구조 개편]\r구간설정 뒤 금액결정 ‘이원화’… 구간설정위원 선정 방...,동아,Con,2019,301,최저 임금 결정 구조 개편 구간 설정 금액 결정 이원 구간 설정 위원 선정 방식 정...,최저 임금 결정 구조 개편 구간 설정 금액 결정 이원 구간 설정 위원 선정 방식 정...,Pro,5
1997,15130,2017-02-23,https://www.hani.co.kr/arti/politics/defense/7...,‘배후’ 드러난 북 “남한이 짠 음모책동”…적반하장 공세,북한 김정은 노동당 위원장의 이복형 김정남 피살 사건에 침묵을 지켜온 평양의 북한 ...,한겨레,Pro,2017,305,북한 김정은 노동당 위원장 이복형 김정남 피살 사건 침묵 평양 북한 당국 공식 반응...,북한 김정은 노동당 위원장 이복형 김정남 피살 침묵 평양 북한 당국 공식 반응 발생...,Pro,3
1998,21848,2023. 02. 10,https://www.khan.co.kr/article/202302101734001,“도이치모터스는 실패한 시세조종”···주가조작 인정되나 ‘집행유예’,2020년 4월 총선을 앞두고 당시 열린민주당 비례대표 후보였던 최강욱 의원 등의 ...,경향,Pro,2023,305,총선 당시 민주당 비례 대표 후보 강욱 의원 고발 수사 시작 도이치 모터스 주가 조...,총선 민주당 비례 대표 후보 강욱 의원 고발 수사 시작 도이치 모터스 주가 조작 선...,Con,9


In [32]:
from joblib import load

vec = load(Path("/content/tfidf.joblib"))
svm_clf = load(Path("/content/svm.joblib"))

In [37]:
# SVM 기준 데이터 변환 및 예측
# ['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']
X = vec.transform(df1["Title"].astype(str))
df1["pred_svm"] = svm_clf.predict(X)

df1

,index,Date,Url,Title,Content,News,Style,Year,Char,Nouns,Nouns_cleaned,Admin,Period\r,pred_svm
0,2050,2019.11.28,https://www.chosun.com/site/data/html_dir/2019...,"[단독] '겨울왕국2' 스크린 2648개↑점유율 79%→디즈니 ""공식입장無""..무법...",[OSEN=김보라 기자] 영화 ‘겨울왕국2’가 1편 ‘겨울왕국’(2014)에 이어 ...,조선,Con,2019,160,김보라 기자 영화 겨울왕국 겨울왕국 관객 돌파 모양새 관객 돌파 평일 일일 관객 극...,김보라 영화 겨울왕국 겨울왕국 관객 돌파 모양새 관객 돌파 평일 일일 관객 극장 겨...,Pro,5,0
1,9626,2018-09-21,https://www.donga.com/news/article/all/2018092...,서울대 총장 재선거에 9명 출사표…오세정 의원 등 등록,바른미래당 오세정 의원을 포함한 9명이 서울대학교 총장 재선거에 출사표를 던졌다. ...,동아,Con,2018,190,미래 오세정 의원 포함 서울대학교 총장 선거 출사표 서울대 총장 최종 후보 성희롱 ...,미래 오세정 의원 포함 서울대학교 총장 선거 출사표 서울대 총장 최종 후보 성희롱 ...,Pro,4,2
2,16339,2024-02-28,https://www.hani.co.kr/arti/society/media/1130...,법원도 ‘청부 민원’ 부당성 인정…방심위 안팎 “류희림 수사해야”,법원이 야권 추천 심의위원에 대한 ‘류희림 방송통신심의위원회(방심위)’와 윤석열 대...,한겨레,Pro,2024,370,법원 야권 추천 심의 위원 류희 방송 통신 심의 위원회 방심위 윤석열 대통령 해촉 ...,법원 야권 추천 심의 위원 류희 방송 통신 심의 위원회 방심위 윤석열 대통령 해촉 ...,Con,10,2
3,6923,2021-07-01,https://www.donga.com/news/article/all/2021070...,공식 등판한 이재명 “억강부약…모두 잘 사는 대동세상 향해가겠다”,더불어민주당 대선 주자인 이재명 경기도지사가 1일 온라인을 통해 제20대 대통령 선...,동아,Con,2021,201,더불어민주당 대선 주자 이재명 경기 도지사 온라인 대통령 선거 출마 선언 이재명 캠...,더불어민주당 대선 주자 이재명 경기 도지사 온라인 대통령 선거 출마 선언 이재명 캠...,Pro,7,3
4,11654,2023-12-06,https://www.donga.com/news/Society/article/all...,채용 과정 개입 후 응시해 팀장 임용…‘채용비리’ 867건 적발,정승윤 국민권익위원회 부위원장 겸 사무처장. 2023.11.22 뉴스1국민권익위원회...,동아,Con,2023,247,정승 국민 권익 위원회 부위원장 사무처장 뉴스1 국민 권익 위원회 공직 유관 단체 ...,정승 국민 권익 위원회 부위원장 사무처장 국민 권익 위원회 공직 유관 단체 공정 채...,Con,9,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,9235,2019-03-04,https://www.donga.com/news/article/all/2019030...,"성인 된 2000년생, ‘공정세대’가 온다",자신과 연관된 불공정에 분노… “큰 성공 어려워” 소박한 삶 지향\r사람 사귈 때도...,동아,Con,2019,315,자신 연관 공정 분노 성공 지향 사람 가성비 동아일보 기성세대 사이 원활 소통 웹뉴...,연관 공정 분노 성공 지향 가성비 동아일보 기성세대 사이 원활 소통 웹뉴 웹툰 뉴스...,Pro,5,0
1996,9358,2019-01-08,https://www.donga.com/news/article/all/2019010...,‘공익위원 추천 정부 독점’ 없앴지만 노사 갈등 더 커질 우려,[최저임금 결정구조 개편]\r구간설정 뒤 금액결정 ‘이원화’… 구간설정위원 선정 방...,동아,Con,2019,301,최저 임금 결정 구조 개편 구간 설정 금액 결정 이원 구간 설정 위원 선정 방식 정...,최저 임금 결정 구조 개편 구간 설정 금액 결정 이원 구간 설정 위원 선정 방식 정...,Pro,5,2
1997,15130,2017-02-23,https://www.hani.co.kr/arti/politics/defense/7...,‘배후’ 드러난 북 “남한이 짠 음모책동”…적반하장 공세,북한 김정은 노동당 위원장의 이복형 김정남 피살 사건에 침묵을 지켜온 평양의 북한 ...,한겨레,Pro,2017,305,북한 김정은 노동당 위원장 이복형 김정남 피살 사건 침묵 평양 북한 당국 공식 반응...,북한 김정은 노동당 위원장 이복형 김정남 피살 침묵 평양 북한 당국 공식 반응 발생...,Pro,3,4
1998,21848,2023. 02. 10,https://www.khan.co.kr/article/202302101734001,“도이치모터스는 실패한 시세조종”···주가조작 인정되나 ‘집행유예’,2020년 4월 총선을 앞두고 당시 열린민주당 비례대표 후보였던 최강욱 의원 등의 ...,경향,Pro,2023,305,총선 당시 민주당 비례 대표 후보 강욱 의원 고발 수사 시작 도이치 모터스 주가 조...,총선 민주당 비례 대표 후보 강욱 의원 고발 수사 시작 도이치 모터스 주가 조작 선...,Con,9,2


In [38]:
# 나이브 베이즈 기준 데이터 변환 및 예측
# ['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치']
X = vec.transform(df1["Title"].astype(str))
df1["pred_nb"] = nb_clf.predict(X)

df1

,index,Date,Url,Title,Content,News,Style,Year,Char,Nouns,Nouns_cleaned,Admin,Period\r,pred_svm,pred_nb
0,2050,2019.11.28,https://www.chosun.com/site/data/html_dir/2019...,"[단독] '겨울왕국2' 스크린 2648개↑점유율 79%→디즈니 ""공식입장無""..무법...",[OSEN=김보라 기자] 영화 ‘겨울왕국2’가 1편 ‘겨울왕국’(2014)에 이어 ...,조선,Con,2019,160,김보라 기자 영화 겨울왕국 겨울왕국 관객 돌파 모양새 관객 돌파 평일 일일 관객 극...,김보라 영화 겨울왕국 겨울왕국 관객 돌파 모양새 관객 돌파 평일 일일 관객 극장 겨...,Pro,5,0,1
1,9626,2018-09-21,https://www.donga.com/news/article/all/2018092...,서울대 총장 재선거에 9명 출사표…오세정 의원 등 등록,바른미래당 오세정 의원을 포함한 9명이 서울대학교 총장 재선거에 출사표를 던졌다. ...,동아,Con,2018,190,미래 오세정 의원 포함 서울대학교 총장 선거 출사표 서울대 총장 최종 후보 성희롱 ...,미래 오세정 의원 포함 서울대학교 총장 선거 출사표 서울대 총장 최종 후보 성희롱 ...,Pro,4,2,2
2,16339,2024-02-28,https://www.hani.co.kr/arti/society/media/1130...,법원도 ‘청부 민원’ 부당성 인정…방심위 안팎 “류희림 수사해야”,법원이 야권 추천 심의위원에 대한 ‘류희림 방송통신심의위원회(방심위)’와 윤석열 대...,한겨레,Pro,2024,370,법원 야권 추천 심의 위원 류희 방송 통신 심의 위원회 방심위 윤석열 대통령 해촉 ...,법원 야권 추천 심의 위원 류희 방송 통신 심의 위원회 방심위 윤석열 대통령 해촉 ...,Con,10,2,2
3,6923,2021-07-01,https://www.donga.com/news/article/all/2021070...,공식 등판한 이재명 “억강부약…모두 잘 사는 대동세상 향해가겠다”,더불어민주당 대선 주자인 이재명 경기도지사가 1일 온라인을 통해 제20대 대통령 선...,동아,Con,2021,201,더불어민주당 대선 주자 이재명 경기 도지사 온라인 대통령 선거 출마 선언 이재명 캠...,더불어민주당 대선 주자 이재명 경기 도지사 온라인 대통령 선거 출마 선언 이재명 캠...,Pro,7,3,6
4,11654,2023-12-06,https://www.donga.com/news/Society/article/all...,채용 과정 개입 후 응시해 팀장 임용…‘채용비리’ 867건 적발,정승윤 국민권익위원회 부위원장 겸 사무처장. 2023.11.22 뉴스1국민권익위원회...,동아,Con,2023,247,정승 국민 권익 위원회 부위원장 사무처장 뉴스1 국민 권익 위원회 공직 유관 단체 ...,정승 국민 권익 위원회 부위원장 사무처장 국민 권익 위원회 공직 유관 단체 공정 채...,Con,9,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,9235,2019-03-04,https://www.donga.com/news/article/all/2019030...,"성인 된 2000년생, ‘공정세대’가 온다",자신과 연관된 불공정에 분노… “큰 성공 어려워” 소박한 삶 지향\r사람 사귈 때도...,동아,Con,2019,315,자신 연관 공정 분노 성공 지향 사람 가성비 동아일보 기성세대 사이 원활 소통 웹뉴...,연관 공정 분노 성공 지향 가성비 동아일보 기성세대 사이 원활 소통 웹뉴 웹툰 뉴스...,Pro,5,0,3
1996,9358,2019-01-08,https://www.donga.com/news/article/all/2019010...,‘공익위원 추천 정부 독점’ 없앴지만 노사 갈등 더 커질 우려,[최저임금 결정구조 개편]\r구간설정 뒤 금액결정 ‘이원화’… 구간설정위원 선정 방...,동아,Con,2019,301,최저 임금 결정 구조 개편 구간 설정 금액 결정 이원 구간 설정 위원 선정 방식 정...,최저 임금 결정 구조 개편 구간 설정 금액 결정 이원 구간 설정 위원 선정 방식 정...,Pro,5,2,4
1997,15130,2017-02-23,https://www.hani.co.kr/arti/politics/defense/7...,‘배후’ 드러난 북 “남한이 짠 음모책동”…적반하장 공세,북한 김정은 노동당 위원장의 이복형 김정남 피살 사건에 침묵을 지켜온 평양의 북한 ...,한겨레,Pro,2017,305,북한 김정은 노동당 위원장 이복형 김정남 피살 사건 침묵 평양 북한 당국 공식 반응...,북한 김정은 노동당 위원장 이복형 김정남 피살 침묵 평양 북한 당국 공식 반응 발생...,Pro,3,4,4
1998,21848,2023. 02. 10,https://www.khan.co.kr/article/202302101734001,“도이치모터스는 실패한 시세조종”···주가조작 인정되나 ‘집행유예’,2020년 4월 총선을 앞두고 당시 열린민주당 비례대표 후보였던 최강욱 의원 등의 ...,경향,Pro,2023,305,총선 당시 민주당 비례 대표 후보 강욱 의원 고발 수사 시작 도이치 모터스 주가 조...,총선 민주당 비례 대표 후보 강욱 의원 고발 수사 시작 도이치 모터스 주가 조작 선...,Con,9,2,4
